In [67]:
from sklearn.mixture import GaussianMixture
import numpy as np
from utils import Givens2Matrix, QRGivens, eigh_with_fixed_direction_range, find_closest_spd
from utils import load_cloud_dataset, load_breast_cancer, load_seg_data, load_digits_dataset, load_satelite_dataset, load_synthetic_dataset
from python_example import Givens2Matrix_double as Givens2Matrix
from python_example import QRGivens_double as QRGivens
from copy import deepcopy
import networkx as nx
from itertools import product

import warnings
from sklearn.exceptions import ConvergenceWarning
# warnings.filterwarnings(action='ignore', category=ComplexWarning)
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [115]:


class EigenParticle:
    def __init__(
        self, 
        n_components,
        data_dim,
        amplitude,
        weights,
        means,
        prec_matrix_list
    ):
        """
        Particle
        """
        self.data_dim = data_dim
        self.n_components = n_components
        self.amplitude = amplitude

        self.velocity = { 
            'weights' : np.zeros(n_components),
            'means' : np.zeros((n_components, data_dim)),
            'eigenvalues_prec' : np.zeros((n_components, data_dim)),
            'givens_angles' : np.zeros((n_components, int(data_dim * (data_dim - 1) / 2)))
        }

        self.position = {
            'weights' : weights,
            'means' : means,
            'eigenvalues_prec' : np.zeros((n_components, data_dim)),
            'givens_angles' : np.zeros((n_components, int(data_dim * (data_dim - 1) / 2)))
        }
        
        self.position['weights'] = weights
        self.position['means'] = means

        for i in range(self.n_components):
            eigenvalues, v = eigh_with_fixed_direction_range(prec_matrix_list[i])
            # eigenvalues, v = np.linalg.eigh(cov_matrix_list[i])
            givens_rotations = QRGivens(v).squeeze()
            
            self.position['eigenvalues_prec'][i] = eigenvalues
            self.position['givens_angles'][i] = givens_rotations

        self.keys = list(self.position.keys())
        
        self.trajectory = [self.position]
        self.person_best = self.position
        self.global_best = self.position
        self.person_best_fitness_score = -np.inf

    def get_cov_matrices(self):
        ret_val = []
        for k in range(self.n_components):
            eigvals = self.position['eigenvalues_prec'][k]
            givens_rotations = self.position['givens_angles'][k]
            v = Givens2Matrix(np.expand_dims(givens_rotations, axis=1))
            cov_matr = v @ np.diag(eigvals) @ v.T
            ret_val.append(cov_matr)

        return ret_val

    def reorder_wrt(self, reference):

        for k in range(self.n_components):  
            eigen_in = self.position['eigenvalues_prec'][k]
            eigen_ref = reference['eigenvalues_prec'][k]
            from python_example import Givens2Matrix_double as Givens2Matrix

            v_ref = np.array(Givens2Matrix(np.expand_dims(reference['givens_angles'][k], axis=1)))
            v_in = np.array(Givens2Matrix(np.expand_dims(self.position['givens_angles'][k], axis=1)))

            eigen_res = []
            v_res = np.zeros_like(v_ref)
        
            untaken_inx = [i for i in range(self.data_dim)]
            permutation = np.zeros([self.data_dim], dtype=np.int32)
            for i in range(self.data_dim):
                i_opt = untaken_inx[np.argmax([abs(np.dot(v_ref[:, i], v_in[:, j])) for j in untaken_inx])]
                untaken_inx.remove(i_opt)
                permutation[i] = i_opt
                
                v_res[i] = deepcopy(v_in[:, i_opt])
                eigen_res.append(deepcopy(eigen_in[i_opt]))

            v_res = v_in[:, permutation]
            eigen_res = eigen_in[permutation]
            
            # print('Norm: ', np.linalg.norm( (v_res @  v_res.T) - (v_in @ v_in.T) ), 'With eig',  np.linalg.norm( (v_res @ np.diag(np.array(eigen_res)) @ v_res.T) - (v_in @ np.diag(eigen_in) @ v_in.T) ))
            
            self.position['eigenvalues_prec'][k] = np.array(eigen_res)

            # from utils import QRGivens
            givens_rotations = QRGivens(np.array(v_res)).squeeze()
            
            self.position['givens_angles'][k] = np.array(givens_rotations)

    def components_reorder(self, reference):
        # reference gloabl best
        G = nx.Graph()

        P1_nodes = [f'P_{i}' for i in range(self.n_components)]
        P2_nodes = [f'ref_{i}' for i in range(self.n_components)]

        G.add_nodes_from(P1_nodes, bipartite=0)
        G.add_nodes_from(P2_nodes, bipartite=1)

        def divergence(mean_1, prec_1, mean_2, prec_2):
            return np.log(np.linalg.det(prec_1) / np.linalg.det(prec_2)) + np.trace(prec_1.dot(np.linalg.inv(prec_2))) + (mean_1 - mean_2).T @ prec_2 @ (mean_1 - mean_2)

        edge_list = list(product([i for i in range(self.n_components)], [i for i in range(self.n_components)]))

        prec_matrcies_1 = np.zeros([self.n_components, self.data_dim, self.data_dim])

        for i in range(self.n_components):
            v = Givens2Matrix(np.expand_dims(self.position['givens_angles'][i], axis=1))
            addition = v @ np.diag(self.position['eigenvalues_prec'][i]) @ v.T
            prec_matrcies_1[i] = addition

        prec_matrcies_2 = np.zeros([self.n_components, self.data_dim, self.data_dim])

        for i in range(self.n_components):
            v = Givens2Matrix(np.expand_dims(reference['givens_angles'][i], axis=1))
            addition = v @ np.diag(reference['eigenvalues_prec'][i]) @ v.T
            prec_matrcies_2[i] = addition

        for edge in edge_list:
            c = divergence(reference['means'][edge[1]], prec_matrcies_2[edge[1]], self.position['means'][edge[0]], prec_matrcies_1[edge[0]])
            G.add_edge(f'ref_{edge[1]}', f'P_{edge[0]}', weight=c)

        matching = nx.algorithms.bipartite.matching.minimum_weight_full_matching(G, P1_nodes)
        permutation = np.zeros(self.n_components, dtype=np.int32)
        for i, node in enumerate(P1_nodes):
            permutation[i] = int(matching[node].split('_')[-1])
        self.position['weights'] = self.position['weights'][permutation]
        self.position['means'] = self.position['means'][permutation]
        self.position['eigenvalues_prec'] = self.position['eigenvalues_prec'][permutation]
        self.position['givens_angles'] = self.position['givens_angles'][permutation]

    def calculate_LL(self, data):
        prec_matrcies = np.zeros([self.n_components, self.data_dim, self.data_dim])
        # prec_matrcies = self.basic_prec_matr

        # construct prec matr addition
        for i in range(self.n_components):
            v = Givens2Matrix(np.expand_dims(self.position['givens_angles'][i], axis=1))
            addition = v @ np.diag(self.position['eigenvalues_prec'][i]) @ v.T
            prec_matrcies[i] = addition
            
        gmm = GaussianMixture(n_components=self.position['weights'].shape[0], covariance_type='full', weights_init=self.position['weights'], means_init=self.position['means'], precisions_init=prec_matrcies, max_iter=100)
        
        cholesky = np.zeros_like(prec_matrcies)
        
        for i in range(self.n_components):
            cholesky[i] = np.linalg.cholesky(prec_matrcies[i])

        gmm.weights_ = self.position['weights']
        gmm.means_ = self.position['means']
        gmm.precisions_cholesky_ = cholesky
        return gmm.score(data)
    
    def _calculate_LL_by_pos(self, data, position):
        prec_matrcies = np.zeros([self.n_components, self.data_dim, self.data_dim])

        # prec_matrcies = self.basic_prec_matr
        # construct prec matr addition
        for i in range(self.n_components):
            v = Givens2Matrix(np.expand_dims(position['givens_angles'][i], axis=1))
            addition = v @ np.diag(position['eigenvalues_prec'][i]) @ v.T
            prec_matrcies[i] = addition
            
        gmm = GaussianMixture(n_components=position['weights'].shape[0], covariance_type='full', weights_init=position['weights'], means_init=position['means'], precisions_init=prec_matrcies, max_iter=100)
        
        cholesky = np.zeros_like(prec_matrcies)
        
        for i in range(self.n_components):
            cholesky[i] = np.linalg.cholesky(prec_matrcies[i])

        gmm.weights_ = position['weights']
        gmm.means_ = position['means']
        gmm.precisions_cholesky_ = cholesky
        return gmm.score(data)
        
    def run_em(self, data, T2):
        self.data = data

        prec_matrcies = np.zeros([self.n_components, self.data_dim, self.data_dim])
        # construct prec matr addition
        for i in range(self.n_components):

            v = Givens2Matrix(np.expand_dims(self.position['givens_angles'][i], axis=1))
            addition = v @ np.diag(self.position['eigenvalues_prec'][i]) @ v.T
            prec_matrcies[i] = addition

        gmm = GaussianMixture(n_components=self.position['weights'].shape[0], covariance_type='full', weights_init=self.position['weights'], means_init=self.position['means'], precisions_init=prec_matrcies, max_iter=T2)
        gmm.fit(data)
        weights = gmm.weights_
        means = gmm.means_
        precisions = gmm.precisions_
        
        self.position['weights'] = weights
        self.position['means'] = means

        for i in range(self.n_components):
            eigenvalues, v = eigh_with_fixed_direction_range(precisions[i])
            # eigenvalues, v = np.linalg.eigh(cov_matrix_list[i])
            givens_rotations = QRGivens(v).squeeze()
            
            self.position['eigenvalues_prec'][i] = eigenvalues
            self.position['givens_angles'][i] = givens_rotations

        self.reorder_wrt(self.person_best)

        if gmm.score(data) > self.person_best_fitness_score:
            self.person_best_fitness_score = gmm.score(data)
            self.person_best = self.position
        
        return gmm.score(data)

    def step(self, c_1, c_2, r_1, r_2):

        for key in self.keys:
            self.velocity[key] = (
                c_1 * r_1[key] * (self.person_best[key] - self.position[key]) + 
                c_2 * r_2[key] * (self.global_best[key] - self.position[key])
            )
            self.position[key] += self.amplitude * self.velocity[key]
        self.trajectory.append(self.position)

        if self.calculate_LL(self.data) > self._calculate_LL_by_pos(self.data, self.person_best):
            print(self.calculate_LL(self.data), self._calculate_LL_by_pos(self.data, self.person_best))
            self.position = self.person_best


class PSOEigen:
    def __init__(self, n_particles, n_comp):
        self.n_particles = n_particles
        self.amplitude = 1
        self.n_components = n_comp

        r_1 = 0.6
        r_2 = 0.8
        r_1_w = 0.42
        r_2_w = 0.57
            
        self.r_1 = {
            'weights' : r_1_w,
            'means' : r_1,
            'eigenvalues_prec' : r_1,
            'givens_angles' : r_1
        }
        
        self.r_2 = {
            'weights' : r_2_w,
            'means' : r_2,
            'eigenvalues_prec' : r_2,
            'givens_angles' : r_2
        }
        self.log_file = 'log_eigen.txt'
        self.global_fitness_score = -np.inf

    def init_global_best(self, data):
        
        self.global_best = self.particles[0]

        global_best_score = self.global_best.calculate_LL(data)

        for p in self.particles:
            score = p.calculate_LL(data)
            if score > global_best_score:
                    global_best_score = score
                    self.global_best = p

        for p in self.particles:
            p.reorder_wrt(self.global_best.position)
            p.components_reorder(self.global_best.position)

    def run(self, data, T1, T2):
        with open(self.log_file, 'w+') as f:
            # make inital GMM
            # scatter points around init GMM
            # contol amplitude of scattering by eigenvalues max values
            # choose global best
            # reorder w.r.t. global best
            data_dim = data[0].shape[0]
            self.particles = []
            for i in range(self.n_particles):
                particle_gmm = GaussianMixture(self.n_components, covariance_type='full', max_iter=1, n_init=1, init_params='k-means++')
                particle_gmm.fit(data)
                self.particles.append(EigenParticle(self.n_components, data_dim, self.amplitude, particle_gmm.weights_, particle_gmm.means_, particle_gmm.precisions_))
                # self.particles = [EigenParticle(self.n_components, self.data[0].shape[0], self.amplitude, weights, means, basic_prec_matr) for i in range(self.n_particles)]

            self.init_global_best(data)

            best_pso_em_score = -np.inf
            best_ll_after_transforn_score = -np.inf

            for i in range(T1):
                f.write(f'Iter {i}\n')
                best_particle = self.particles[0]
                for j in range(len(self.particles)):
                    print('Before EM: ', self.particles[j].calculate_LL(data))
                    new_ll = self.particles[j].run_em(data, T2)
                    print('After EM: ', self.particles[j].calculate_LL(data))

                    f.write('New LL: ' + str(new_ll) + '\n')
                    if best_pso_em_score < new_ll:
                        best_pso_em_score = new_ll
                        best_particle = self.particles[j]

                changed = False
                for particle in self.particles:
                    if particle.person_best_fitness_score > self.global_fitness_score:
                        self.global_fitness_score = particle.person_best_fitness_score
                        self.global_best = deepcopy(particle)
                        changed = True

                for p in self.particles:
                    p.global_best = self.global_best.position

                if changed:
                    for particle in self.particles:
                        pass
                        print('Before reorder', particle.calculate_LL(data))
                        particle.reorder_wrt(self.global_best.position)
                        particle.components_reorder(self.global_best.position)

                        print('after reorder', particle.calculate_LL(data))

                for i in range(1):
                    c_1 = np.random.uniform(0, 1)
                    c_2 = np.random.uniform(0, 1)
                    for i in range(len(self.particles)):
                        self.particles[i].step(c_1, c_2, self.r_1, self.r_2)
                    f.flush()

                    changed = False
                    for particle in self.particles:
                        if particle.person_best_fitness_score > self.global_fitness_score:
                            self.global_fitness_score = particle.person_best_fitness_score
                            self.global_best = deepcopy(particle)
                            changed = True

                    for p in self.particles:
                        p.global_best = self.global_best.position
                    
                if changed:
                    for particle in self.particles:
                        pass
                        particle.reorder_wrt(self.global_best.position)
                        particle.components_reorder(self.global_best.position)

            f.write('Personal bests:')
            for particle in self.particles:
                f.write(str(particle.person_best_fitness_score))
            f.write(f'\n Global best: {self.global_fitness_score}')

        return 



In [116]:
data = load_seg_data()
T1 = 50
T2 = 20
n_comp = 10
n_particles = 50

em = PSOEigen(n_particles, n_comp)

em.run(data, T1, T2)

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")

Before EM:  65.30102847174062
After EM:  65.8871474396805
Before EM:  65.20349752935658
After EM:  67.18148380026659
Before EM:  64.83521105299589
After EM:  66.7514761515702
Before EM:  67.56792564293555
After EM:  68.91356943902801
Before EM:  64.66784974378085
After EM:  66.54473919336415
Before EM:  66.04261040563458
After EM:  68.35883059288817
Before EM:  65.65181725076387
After EM:  66.7872676808744
Before EM:  68.97392387512974
After EM:  70.23453911463886
Before EM:  67.1676921601287
After EM:  69.22181821095371
Before EM:  67.16246694654431
After EM:  68.35106281071083
Before EM:  64.75062665000834
After EM:  66.79274142843634
Before EM:  65.08470062652225
After EM:  65.9596999534644
Before EM:  63.59527883961005
After EM:  65.72051313158363
Before EM:  64.57827143190879
After EM:  68.44201991612812
Before EM:  67.54747310313385
After EM:  69.21769428787175
Before EM:  67.30396627361588
After EM:  68.5538612999664
Before EM:  65.96859155459696
After EM:  68.32935922719494
Bef

KeyboardInterrupt: 